In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

In [ ]:
base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','origin','destination','primary_purpose','person_id','start','end','tour_type'])

base_tour_idx = base_tour.index

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','origin','destination','primary_purpose','person_id','start','end','tour_type'])


In [ ]:
print(f"""Filtering tours by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination TAZ.\nOriginal tours in base: {len(base_tour)}\tbuild: {len(build_tour)}\tdiff: {len(build_tour)-len(base_tour)}""")

base_tour = base_tour[multi_filter_mazs([base_tour.origin, base_tour.destination])] # base tour in the filtered set

# special build-case filtering
build_tour = build_tour[(
    build_tour.index.isin(base_tour_idx) # originally existed in base
    & build_tour.index.isin(base_tour.index)    # and was in the current set for the base
    ) |     # OR
    ((~build_tour.index.isin(base_tour_idx)) # is a new tour 
     & multi_filter_mazs([build_tour.origin, build_tour.destination]) # and it's in this set
     )]

print(f"After filtering, tours in base: {len(base_tour)}\tbuild: {len(build_tour)}\tdiff: {len(build_tour)-len(base_tour)}")

In [ ]:
name_dict = {'escort': "Escorting", 
             'work': "Work", 
             'school': "School", 
             'othmaint': "Other-Maintenance",
             'othdiscr': "Other-Discretionary", 
             'shopping': "Shopping",
             'atwork': "At-work Subtour", 
             'eatout': "Eat Out", 
             'social' : "Social", 
             'univ': "University"}

## Non-Mandatory Tour Destinations

In [ ]:
df = base_tour[base_tour.tour_category == 'non_mandatory'].merge(build_tour[build_tour.tour_category == 'non_mandatory'],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))



In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key, value in name_dict.items():

    if len(base_tour[
        (base_tour.tour_category == 'non_mandatory')
        & (base_tour.primary_purpose == key)
        ]) == 0:
        continue
    
    print(f"#### {value}")
    df = base_tour[
        (base_tour.tour_category == 'non_mandatory')
        & (base_tour.primary_purpose == key)
        ].merge(build_tour[
                    (build_tour.tour_category == 'non_mandatory')
                    & (build_tour.primary_purpose == key)
                ],
                how='left',
                left_index=True,
                right_index=True,
                suffixes=('_base','_build'))
    
    df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
    df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

    df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
    df.loc[df.destination_build.isna(),"Case"] = "Removed"

    
    df = df.Case.value_counts()

    print(str(df).replace("\n","<br>"))
    
    fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
    
    fig.show()
    display(Markdown(" "))

## Mandatory Tour Scheduling

In [ ]:
# keep only work and school tours
base_tours_df = base_tour[base_tour.tour_type.isin(['work', 'school'])]
base_tour_idx = base_tours_df.index

build_tours_df = build_tour[build_tour.tour_type.isin(['work', 'school'])]

In [ ]:
# translate time bin to time period index
base_tours_df['start_period'] = base_tours_df.start.apply(vh.get_time_period_index)
build_tours_df['start_period'] = build_tours_df.start.apply(vh.get_time_period_index)

base_tours_df['end_period'] = base_tours_df.end.apply(vh.get_time_period_index)
build_tours_df['end_period'] = build_tours_df.end.apply(vh.get_time_period_index)

In [ ]:
# join base and build tours on tour_id
# compare tours that are in both base and build datasets
df = pd.merge(
    base_tours_df,
    build_tours_df,
    left_index=True,
    right_index=True,
    suffixes=('_base', '_build'),
    how='inner'
)

# get the difference in start and end times
df['start_bin_difference'] = df.start_build - df.start_base
df['end_bin_difference'] = df.end_build - df.end_base

# get the difference in start and end time periods
df['start_period_difference'] = df.start_period_build - df.start_period_base
df['end_period_difference'] = df.end_period_build - df.end_period_base

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for purpose in ['work', 'school']:
    for metric in ['start', 'end']:
        purpose_df = df[df.tour_type_base == purpose]
        display(Markdown(f"### {purpose.capitalize()} Tour {metric.capitalize()} Time Bin Difference"))
        purpose_df = purpose_df.groupby(f'{metric}_bin_difference').size().reset_index()
        purpose_df.columns = [f'{metric}_bin_difference', 'count']
        purpose_df = purpose_df.sort_values(by=f'{metric}_bin_difference')

        fig = vh.create_bar_chart(
            source_data=purpose_df,
            source=f'{metric}_bin_difference',
            col=f'{metric}_bin_difference',
            plot_col='count',
            # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
        )
        fig.show()
        display(Markdown(" "))

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for purpose in ['work', 'school']:
    for metric in ['start', 'end']:
        purpose_df = df[df.tour_type_base == purpose]
        display(Markdown(f"### {purpose.capitalize()} Tour {metric.capitalize()} Time Period Difference"))
        purpose_df = purpose_df.groupby(f'{metric}_period_difference').size().reset_index()
        purpose_df.columns = [f'{metric}_period_difference', 'count']
        purpose_df = purpose_df.sort_values(by=f'{metric}_period_difference')

        fig = vh.create_bar_chart(
            source_data=purpose_df,
            source=f'{metric}_period_difference',
            col=f'{metric}_period_difference',
            plot_col='count',
            # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
        )
        fig.show()
        display(Markdown(" "))

## Non-Mandatory Tour Scheduling

In [ ]:
# start with the filtered tours, get non-mandatory tours only
base_tours_df = base_tour[base_tour.tour_category == 'non_mandatory']
build_tours_df = build_tour[build_tour.tour_category == 'non_mandatory']

In [ ]:
# translate time bin to time period index
base_tours_df['start_period'] = base_tours_df.start.apply(vh.get_time_period_index)
build_tours_df['start_period'] = build_tours_df.start.apply(vh.get_time_period_index)

base_tours_df['end_period'] = base_tours_df.end.apply(vh.get_time_period_index)
build_tours_df['end_period'] = build_tours_df.end.apply(vh.get_time_period_index)

In [ ]:
# join base and build tours on tour_id
# compare tours that are in both base and build datasets
df = pd.merge(
    base_tours_df,
    build_tours_df,
    left_index=True,
    right_index=True,
    suffixes=('_base', '_build'),
    how='inner'
)

# get the difference in start and end times
df['start_bin_difference'] = df.start_build - df.start_base
df['end_bin_difference'] = df.end_build - df.end_base

# get the difference in start and end time periods
df['start_period_difference'] = df.start_period_build - df.start_period_base
df['end_period_difference'] = df.end_period_build - df.end_period_base

### Tour Start Time Bin Difference

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key, value in name_dict.items():
    purpose_df = df[df.primary_purpose_base == key]
    if len(purpose_df) == 0:
        continue
    display(Markdown(f"### {value.capitalize()}"))
    purpose_df = purpose_df.groupby(f'start_bin_difference').size().reset_index()
    purpose_df.columns = [f'start_bin_difference', 'count']
    purpose_df = purpose_df.sort_values(by=f'start_bin_difference')

    fig = vh.create_bar_chart(
        source_data=purpose_df,
        source=f'start_bin_difference',
        col=f'start_bin_difference',
        plot_col='count',
        # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
    )
    fig.show()
    display(Markdown(" "))

### Tour Start Time Period Difference

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key, value in name_dict.items():
    purpose_df = df[df.primary_purpose_base == key]
    if len(purpose_df) == 0:
        continue
    display(Markdown(f"### {value.capitalize()}"))
    purpose_df = purpose_df.groupby(f'start_period_difference').size().reset_index()
    purpose_df.columns = [f'start_period_difference', 'count']
    purpose_df = purpose_df.sort_values(by=f'start_period_difference')

    fig = vh.create_bar_chart(
        source_data=purpose_df,
        source=f'start_period_difference',
        col=f'start_period_difference',
        plot_col='count',
        # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
    )
    fig.show()
    display(Markdown(" "))

### Tour End Time Bin Difference

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key, value in name_dict.items():
    purpose_df = df[df.primary_purpose_base == key]
    if len(purpose_df) == 0:
        continue
    display(Markdown(f"### {value.capitalize()}"))
    purpose_df = purpose_df.groupby(f'end_bin_difference').size().reset_index()
    purpose_df.columns = [f'end_bin_difference', 'count']
    purpose_df = purpose_df.sort_values(by=f'end_bin_difference')

    fig = vh.create_bar_chart(
        source_data=purpose_df,
        source=f'end_bin_difference',
        col=f'end_bin_difference',
        plot_col='count',
        # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
    )
    fig.show()
    display(Markdown(" "))

### Tour End Period Difference

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key, value in name_dict.items():
    purpose_df = df[df.primary_purpose_base == key]
    if len(purpose_df) == 0:
        continue
    display(Markdown(f"### {value.capitalize()}"))
    purpose_df = purpose_df.groupby(f'end_period_difference').size().reset_index()
    purpose_df.columns = [f'end_period_difference', 'count']
    purpose_df = purpose_df.sort_values(by=f'end_period_difference')

    fig = vh.create_bar_chart(
        source_data=purpose_df,
        source=f'end_period_difference',
        col=f'end_period_difference',
        plot_col='count',
        # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
    )
    fig.show()
    display(Markdown(" "))

## Tour Mode Choice

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## At-Work Subtour Frequency

In [ ]:
# at-work subtours are decisions made by work tours, so we need to filter based on the work location

# read in unfiltered tours for base and build

base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','primary_purpose','destination', 'parent_tour_id'])

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','primary_purpose','destination', 'parent_tour_id'])

# keep work tours only
base_work_tour = base_tour[base_tour.primary_purpose == 'work']
build_work_tour = build_tour[build_tour.primary_purpose == 'work']
base_work_tour_idx = base_work_tour.index


In [ ]:
print(f"""Filtering work tours by destination MAZ.\nOriginal tours in base: {len(base_work_tour)}\tbuild: {len(build_work_tour)}\tdiff: {len(build_work_tour)-len(base_work_tour)}""")

base_work_tour = base_work_tour[single_filter_mazs(base_work_tour.destination)] # base tour in the filtered set

# special build-case filtering
build_work_tour = build_work_tour[(
    build_work_tour.index.isin(base_work_tour_idx) # originally existed in base
    & build_work_tour.index.isin(base_work_tour.index)    # and was in the current set for the base
    ) |     # OR
    ((~build_work_tour.index.isin(base_work_tour_idx)) # is a new tour 
     & single_filter_mazs(build_work_tour.destination)# and it's in this set
     )]

print(f"After filtering, tours in base: {len(base_work_tour)}\tbuild: {len(build_work_tour)}\tdiff: {len(build_work_tour)-len(base_work_tour)}")

In [ ]:
# now get the at-work subtours from the unfiltered tours
base_atwork_tours_df = base_tour[base_tour.primary_purpose == 'atwork']
build_atwork_tours_df = build_tour[build_tour.primary_purpose == 'atwork']

In [ ]:
# now count at-work subtours by parent_tour_id
base_atwork_tour_counts_df = base_atwork_tours_df.groupby('parent_tour_id').size()
build_atwork_tour_counts_df = build_atwork_tours_df.groupby('parent_tour_id').size()
base_atwork_tour_counts_df = base_atwork_tour_counts_df.rename('at-work')
build_atwork_tour_counts_df = build_atwork_tour_counts_df.rename('at-work')

# reindex to ensure all work tours are included, filling missing with 0
base_work_atwork_counts_df = base_atwork_tour_counts_df.reindex(base_work_tour.index, fill_value=0)
build_work_atwork_counts_df = build_atwork_tour_counts_df.reindex(build_work_tour.index, fill_value=0)

display(Markdown(f"### At-Work Subtour Frequency\n"))
df = pd.crosstab(
    base_work_atwork_counts_df, 
    build_work_atwork_counts_df, 
    rownames=['Base'], 
    colnames=['Build'], 
    margins=True, 
    margins_name='Total'
)
df

## At-Work Subtour Destinations

In [ ]:
base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','origin','destination','primary_purpose','start','end'])

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','origin','destination','primary_purpose','start','end'])


In [ ]:
print(f"""Filtering tours by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination TAZ.\nOriginal tours in base: {len(base_tour)}\tbuild: {len(build_tour)}\tdiff: {len(build_tour)-len(base_tour)}""")

base_tour = base_tour[multi_filter_mazs([base_tour.origin, base_tour.destination])] # base tour in the filtered set

# special build-case filtering
build_tour = build_tour[(
    build_tour.index.isin(base_tour_idx) # originally existed in base
    & build_tour.index.isin(base_tour.index)    # and was in the current set for the base
    ) |     # OR
    ((~build_tour.index.isin(base_tour_idx)) # is a new tour 
     & multi_filter_mazs([build_tour.origin, build_tour.destination]) # and it's in this set
     )]

print(f"After filtering, tours in base: {len(base_tour)}\tbuild: {len(build_tour)}")

In [ ]:
df = base_tour[base_tour.tour_category == 'atwork'].merge(build_tour[build_tour.tour_category == 'atwork'],
                                                          how='outer',
                                                          left_index=True,
                                                          right_index=True,
                                                          suffixes=('_base','_build'))


In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## At-Work Subtour Scheduling

In [ ]:
base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','origin','destination','primary_purpose','start','end'])
base_tour = base_tour[base_tour.tour_category == 'atwork']

base_tour_idx = base_tour.index

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','origin','destination','primary_purpose','start','end'])
build_tour = build_tour[build_tour.tour_category == 'atwork']

In [ ]:
print(f"""Filtering tours by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination TAZ.\nOriginal tours in base: {len(base_tour)}\tbuild: {len(build_tour)}\tdiff: {len(build_tour)-len(base_tour)}""")

base_tour = base_tour[multi_filter_mazs([base_tour.origin, base_tour.destination])] # base tour in the filtered set

# special build-case filtering
build_tour = build_tour[(
    build_tour.index.isin(base_tour_idx) # originally existed in base
    & build_tour.index.isin(base_tour.index)    # and was in the current set for the base
    ) |     # OR
    ((~build_tour.index.isin(base_tour_idx)) # is a new tour 
     & multi_filter_mazs([build_tour.origin, build_tour.destination]) # and it's in this set
     )]

print(f"After filtering, tours in base: {len(base_tour)}\tbuild: {len(build_tour)}")

In [ ]:
# start with the filtered tours
# base_atwork_tours_df = base_tour[base_tour.tour_category == 'atwork']
# build_atwork_tours_df = build_tour[build_tour.tour_category == 'atwork']

# translate time bin to time period index
base_tour['start_period'] = base_tour.start.apply(vh.get_time_period_index)
build_tour['start_period'] = build_tour.start.apply(vh.get_time_period_index)
base_tour['end_period'] = base_tour.end.apply(vh.get_time_period_index)
build_tour['end_period'] = build_tour.end.apply(vh.get_time_period_index) 

In [ ]:
# join base and build tours on tour_id
# compare tours that are in both base and build datasets
df = pd.merge(
    base_tour,
    build_tour,
    left_index=True,
    right_index=True,
    suffixes=('_base', '_build'),
    how='inner'
)

# get the difference in start and end times
df['start_bin_difference'] = df.start_build - df.start_base
df['end_bin_difference'] = df.end_build - df.end_base

# get the difference in start and end time periods
df['start_period_difference'] = df.start_period_build - df.start_period_base
df['end_period_difference'] = df.end_period_build - df.end_period_base

In [ ]:
display(Markdown(f"### Atwork Subtour Start Time Bin Difference"))
purpose_df = df.groupby(f'start_bin_difference').size().reset_index()
purpose_df.columns = [f'start_bin_difference', 'count']
purpose_df = purpose_df.sort_values(by=f'start_bin_difference')

fig = vh.create_bar_chart(
    source_data=purpose_df,
    source=f'start_bin_difference',
    col=f'start_bin_difference',
    plot_col='count',
    # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
)
fig.show()
display(Markdown(" "))

In [ ]:
display(Markdown(f"### Atwork Subtour Start Time Period Difference"))
purpose_df = df.groupby(f'start_period_difference').size().reset_index()
purpose_df.columns = [f'start_period_difference', 'count']
purpose_df = purpose_df.sort_values(by=f'start_period_difference')

fig = vh.create_bar_chart(
    source_data=purpose_df,
    source=f'start_period_difference',
    col=f'start_period_difference',
    plot_col='count',
    # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
)
fig.show()
display(Markdown(" "))

In [ ]:
display(Markdown(f"### Atwork Subtour End Time Bin Difference"))
purpose_df = df.groupby(f'end_period_difference').size().reset_index()
purpose_df.columns = [f'end_period_difference', 'count']
purpose_df = purpose_df.sort_values(by=f'end_period_difference')

fig = vh.create_bar_chart(
    source_data=purpose_df,
    source=f'end_period_difference',
    col=f'end_period_difference',
    plot_col='count',
    # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
)
fig.show()
display(Markdown(" "))

In [ ]:
display(Markdown(f"### Atwork Subtour End Time Period Difference"))
purpose_df = df.groupby(f'end_period_difference').size().reset_index()
purpose_df.columns = [f'end_period_difference', 'count']
purpose_df = purpose_df.sort_values(by=f'end_period_difference')

fig = vh.create_bar_chart(
    source_data=purpose_df,
    source=f'end_period_difference',
    col=f'end_period_difference',
    plot_col='count',
    # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
)
fig.show()
display(Markdown(" "))

## At-Work Subtour Mode Choice

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Stop Frequency

In [ ]:
print("Lorem ipsum") # Placeholder for CS